Some of this code is Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [3]:
# Uncomment the below line to download the dataset (approx. 2gb)
# !curl -O https://storage.googleapis.com/nasbench/nasbench_full.tfrecord

In [4]:
# Initialize the NASBench object which parses the raw data into memory
# This may take a few minutes, about 2 on my machine
import copy
import math
import random
import numpy as np
import pandas as pd
from nasbench import api
from typing import List, Set, Union
from nasbench.api import ModelSpec
from dataclasses import dataclass
from functools import lru_cache
from IPython.display import clear_output

In [5]:
nasbench = api.NASBench("nasbench_full.tfrecord")

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 126 seconds


In [6]:
INPUT = "input"
OUTPUT = "output"
CONV3X3 = "conv3x3-bn-relu"
CONV1X1 = "conv1x1-bn-relu"
MAXPOOL3X3 = "maxpool3x3"

NUM_VERTICES = 7
MAX_EDGES = 9

EDGE_SPOTS = NUM_VERTICES * (NUM_VERTICES - 1) / 2  # Upper triangular matrix
OP_SPOTS = NUM_VERTICES - 2  # Input/output vertices are fixed

ALLOWED_OPS = [CONV3X3, CONV1X1, MAXPOOL3X3]
ALLOWED_EDGES = [0, 1]  # Binary adjacency matrix

RNG_SEED = 42

In [7]:
@dataclass
class ModelData:
    """
    Wraps up resulting data from NASBench.query in a class for code cleanliness.
    """
    hash: str
    matrix: List[List[int]]
    operations: List[str]
    parameters: int
    train_time: float
    train_accuracy: float
    valid_accuracy: float
    test_accuracy: float


class SpecWrapper(ModelSpec):
    """
    Wraps a model to allow easier access to the resulting data of the model.
    """
    def __init__(self, matrix: Union[str, np.ndarray], ops: List[str] = None):
        if isinstance(matrix, str):
            spec = nasbench.get_metrics_from_hash(matrix)
            matrix: np.ndarray = spec[0]["module_adjacency"]
            ops = spec[0]["module_operations"]
            super().__init__(matrix, ops)
        else:
            assert ops is not None
            super().__init__(matrix, ops)

    def __lt__(self, other: "SpecWrapper"):
        if not (isinstance(other, SpecWrapper)):
            return False

        return self.get_data().test_accuracy < other.get_data().test_accuracy

    def get_hash(self):
        return self.hash_spec(nasbench.config["available_ops"])

    @lru_cache(maxsize=1)
    def get_data(self):
        """
        Get resultant data from NASBench.
        The LRU cache ensures we don"t add time to the budget counters multiple times.
        :return:
        """
        data = nasbench.query(self)
        return ModelData(
            self.hash_spec(nasbench.config["available_ops"]),
            data["module_adjacency"],
            data["module_operations"],
            data["trainable_parameters"],
            data["training_time"],
            data["train_accuracy"],
            data["validation_accuracy"],
            data["test_accuracy"]
        )

    def __repr__(self):
        return self.get_hash()

In [8]:
all_specs = list(nasbench.hash_iterator())

In [9]:
def get_spec(spec_hash: str) -> Union[SpecWrapper, None]:
    """
    Fetch a spec based on the hash.
    :param spec_hash: The hash of the spec.
    :return: A SpecWrapper of the spec or None if it doesn't exist.
    """
    if not spec_hash in all_specs:
        return None

    spec = nasbench.get_metrics_from_hash(spec_hash)
    matrix: np.ndarray = spec[0]["module_adjacency"]
    ops: List[str] = spec[0]["module_operations"]
    return SpecWrapper(matrix=matrix, ops=ops)

In [10]:
def random_spec() -> SpecWrapper:
    """
    Return a random spec.
    :return: A SpecWrapper for the spec.
    """
    return get_spec(random.choice(all_specs))

In [11]:
def sel_best(population: List[SpecWrapper], k: int) -> List[SpecWrapper]:
    """
    Select the top k candidates amongst the entire population.
    :param population: The population to select from.
    :param k: The number of top individuals to select.
    :return: The selected individuals.
    """
    # clone the list so we don't modify the passed list
    population = list(population)
    population.sort()
    return population[-k:]


def sel_random(population: List[SpecWrapper], k: int) -> List[SpecWrapper]:
    """
    Select k individuals from the population at random.
    :param population: The population to select from.
    :param k: The number of individuals to select.
    :return: The selected individuals.
    """
    return random.sample(population, k)


def sel_tournament(population: List[SpecWrapper], k: int, n: int) -> List[SpecWrapper]:
    """
    Select the n best individuals from amongst k randomly selected candidates.
    :param population: The population to select from.
    :param k: The number of candidates to randomly select.
    :param n: The number of top candidates to be returned.
    :return: The selected individuals.
    """
    candidates = sel_random(population, k)
    return sel_best(candidates, n)


def drop_worst(population: List[SpecWrapper], k: int) -> List[SpecWrapper]:
    """
    Drop the k worst candidates and return the remaining population.
    :param population: The population to drop from.
    :param k: Number of worst candidates to drop.
    """
    # clone the list so we don't modify the passed list
    population = list(population)
    population.sort()
    return population[k:]



In [12]:
def make_specs(population: List[str]) -> List[SpecWrapper]:
    return [SpecWrapper(ind) for ind in population]

In [13]:
def run_random_search(k: int, max_time: float):
    """
    Runs a simulated random search for a fixed amount of time. Actual time taken will be
    relatively quick, only the simulated time is considered. Will return all evaluated models,
    along with the k best models found.
    :param k: Number of best models to return
    :param max_time_budget: The amount of simulated time to expend.
    :return: All models evaluated, along with the best k models.
    """
    # resetting seeds at the top of the search functions is required for reproducibility
    np.random.seed(RNG_SEED)
    random.seed(RNG_SEED)

    # nasbench tracks time taken with each query, effectively simulating full
    # training, as we can query how long into the training we would be if we
    # were training ourselves. we reset the counters at the top of each experiment
    nasbench.reset_budget_counters()

    best: List[SpecWrapper] = []
    models: List[SpecWrapper] = []
    evaluated: Set[str] = set()
    collisions: int = 0

    time_spent, _ = nasbench.get_budget_counters()
    while time_spent < max_time:
        spec = random_spec()
        spec_hash = spec.hash_spec(nasbench.config["available_ops"])

        if spec_hash in evaluated:
            collisions += 1
            continue
        else:
            evaluated.add(spec_hash)

        models.append(spec)
        best = sel_best(models, k)
        time_spent, _ = nasbench.get_budget_counters()
        clear_output(wait=True)
        print(f"{time_spent/1000:0.2f}/{max_time/1000:0.0f}k ({(time_spent/max_time)*100:0.2f}%) seconds simulated")

    print(f"{len(evaluated)} unique models during random search.")
    print(f"{collisions} collisions during random search.")

    return models, best



In [14]:
all, best = run_random_search(10, 5e8)

503537.68/500000k (100.71%) seconds simulated
218 unique models during random search.
0 collisions during random search.


In [15]:
adf = pd.DataFrame(list(map(lambda x: x.get_data(), all)))
adf = adf.drop(columns=['matrix', 'operations'])
adf.sort_values(by=["test_accuracy"], ascending=False)

,hash,parameters,train_time,train_accuracy,valid_accuracy,test_accuracy
193,49c5cc265b8d4f35853c68d24a4ce4b7,30515338,3857.607910,1.000000,0.944111,0.938101
169,a39471ea746575cf09d6e7cc1d2970d2,8555530,1621.181885,1.000000,0.938201,0.937099
70,939b5bc9edf798f9acd09345be8a52fb,15747722,2963.628906,1.000000,0.941206,0.936398
113,fbad25518689a2aa5aa6490ce8a6c74b,6230410,1678.697021,1.000000,0.939103,0.933694
142,8ab47dc234a78005f41ca30b281a6ffa,31552906,4087.083984,1.000000,0.936899,0.932893
...,...,...,...,...,...,...
82,f05919dd1f6eea185c4b5616ae269b9a,535071,818.460999,0.985978,0.846554,0.838441
56,5990289a47a65df2afd4fac6cb0174fb,5906570,1860.664062,0.976663,0.829427,0.831230
210,c20f2084c5eef883371da62b19164780,2957706,1606.395996,0.975060,0.818109,0.805389
153,7d6b0743b6f0afbc8fc3627f3836691b,6944138,1692.015991,0.683794,0.655148,0.655549


In [16]:
bdf = pd.DataFrame(list(map(lambda x: x.get_data(), best)))
bdf = bdf.drop(columns=['matrix', 'operations'])
bdf.sort_values(by=["test_accuracy"], ascending=False)

,hash,parameters,train_time,train_accuracy,valid_accuracy,test_accuracy
2,49c5cc265b8d4f35853c68d24a4ce4b7,30515338,3857.607910,1.0,0.944111,0.938101
9,a39471ea746575cf09d6e7cc1d2970d2,8555530,1621.181885,1.0,0.938201,0.937099
8,939b5bc9edf798f9acd09345be8a52fb,15747722,2963.884033,1.0,0.941206,0.934696
7,8ab47dc234a78005f41ca30b281a6ffa,31552906,4090.587891,1.0,0.939804,0.933093
1,bf9c69ec3e4bdfcf4b4c1b1e3d784b8d,15911562,2940.676025,1.0,0.941807,0.932893
5,af6bc6497d28a2ac0d07401faf78d782,10183050,1660.737061,1.0,0.937800,0.932191
3,9688103ecb4c9130f28a8640155b46a8,4206540,1458.468994,1.0,0.936899,0.931390
4,611666cc4f9fa529c7321b3f7c4dce09,15037834,2505.226074,1.0,0.933894,0.929487
6,5ee3351fc0effac626dcdd1d5d8d1359,8816266,1741.989990,1.0,0.935096,0.928786
0,9d365737520fe9c57442e1d1c1ed2842,4426762,1239.964966,1.0,0.931190,0.928586


In [17]:
def crossover_spec(spec_a: SpecWrapper, spec_b: SpecWrapper, cxprb: float = 0.5) -> SpecWrapper:
    assert spec_a.matrix.shape == spec_b.matrix.shape

    while True:
        new_matrix = copy.deepcopy(spec_a.matrix)
        new_ops = copy.deepcopy(spec_a.ops)

        for ridx, row in enumerate(spec_b.matrix):
            for cidx, itm in enumerate(row):
                if random.random() < cxprb:
                    new_matrix[ridx][cidx] = itm

        for oidx, op in enumerate(spec_b.ops):
            if random.random() < cxprb:
                new_ops[oidx] = op

        new_spec = SpecWrapper(new_matrix, new_ops)
        if nasbench.is_valid(new_spec):
            return new_spec



In [26]:
def mutate_spec(old_spec, mutation_rate=1.0):
    """Computes a valid mutated spec from the old_spec."""
    old_size = old_spec.matrix.shape[0]
    while True:
        new_matrix = copy.deepcopy(old_spec.original_matrix)
        new_ops = copy.deepcopy(old_spec.original_ops)

        # In expectation, V edges flipped (note that most end up being pruned).
        edge_mutation_prob = mutation_rate / old_size
        for src in range(0, old_size - 1):
            for dst in range(src + 1, old_size):
                if random.random() < edge_mutation_prob:
                    new_matrix[src, dst] = 1 - new_matrix[src, dst]

        # In expectation, one op is resampled.
        op_mutation_prob = mutation_rate / OP_SPOTS
        for ind in range(1, old_size - 1):
            if random.random() < op_mutation_prob:
                available = [
                    o for o in nasbench.config["available_ops"] if o != new_ops[ind]
                ]
                new_ops[ind] = random.choice(available)

        new_spec = SpecWrapper(new_matrix, new_ops)
        if nasbench.is_valid(new_spec):
            return new_spec


def run_evolutionary_search(max_time: float, pop_size: int, n_best: int):
    # resetting seeds at the top of the search functions is required for reproducibility
    np.random.seed(RNG_SEED)
    random.seed(RNG_SEED)

    # nasbench tracks time taken with each query, effectively simulating full
    # training, as we can query how long into the training we would be if we
    # were training ourselves. we reset the counters at the top of each experiment
    nasbench.reset_budget_counters()

    population = [get_spec(ind) for ind in sel_random(all_specs, pop_size)]
    [ind.get_data() for ind in population]

    best: List[SpecWrapper] = sel_best(population, n_best)
    evaluated: Set[str] = set(map(lambda x: x.get_hash(), population))

    def update_evaluated(new_items: List[SpecWrapper]):
        [evaluated.add(ind.get_hash()) for ind in new_items]

    time_spent, _ = nasbench.get_budget_counters()
    while time_spent < max_time:
        [ind.get_data() for ind in population]

        n_new = math.ceil(len(population)*0.05)
        population = drop_worst(population, n_new)

        new_specs: List[SpecWrapper] = []
        while len(new_specs) < n_new:
            candidate = sel_tournament(population, 25, 1)[0]
            candidate = mutate_spec(candidate, 1)
            cand_hash = candidate.get_hash()

            if cand_hash in evaluated:
                continue

            update_evaluated([candidate])
            candidate.get_data()
            new_specs.append(candidate)

        population = [*population, *new_specs]
        time_spent, _ = nasbench.get_budget_counters()
        best = sel_best(population, n_best)
        best.sort()

        clear_output(wait=True)
        abs_best = best[-1].get_data()
        print(f"{time_spent/1000:0.2f}/{max_time/1000:0.0f}k ({(time_spent/max_time)*100:0.2f}%) seconds simulated")
        print(f'Best so far -- Test: {abs_best.test_accuracy:0.7f}, Valid: {abs_best.valid_accuracy:0.7f}')

    return population, best, evaluated

population, best, all = run_evolutionary_search(5e8, 100, 10)

514408.12/500000k (102.88%) seconds simulated
Best so far -- Test: 0.9403045, Valid: 0.9445112


In [27]:
adf = pd.DataFrame(list(map(lambda x: x.get_data(), population)))
adf = adf.drop(columns=['matrix', 'operations'])
adf.sort_values(by=["test_accuracy"], ascending=False)

,hash,parameters,train_time,train_accuracy,valid_accuracy,test_accuracy
94,1cd3075dacd68bc6fd43ba6d2870a431,23131530,3159.622070,1.0,0.945713,0.944311
84,a5af90a9ffbb95c7737f9e47af569c04,31552906,3630.953125,1.0,0.945312,0.943409
92,f5d9849d3c37a810ac43b8a42aa5d19f,23131530,2967.960938,1.0,0.942508,0.942308
51,2bf4e3dbd48d84126f7ac1862e18752d,30515338,3356.468994,1.0,0.944611,0.941406
65,1beb83aa184aa5720b498e9636abc089,32426634,3913.979004,1.0,0.947115,0.941206
...,...,...,...,...,...,...
98,caeaaad4ab2c04519abf3652ac406992,14000266,2523.416016,1.0,0.934896,0.929187
7,cf3944bc50c7ed89e9b9ce89646d26f1,23295370,2991.322021,1.0,0.938301,0.927584
2,01d32ce90de0879a63bd14cede790fea,15911562,2516.398926,1.0,0.934996,0.925881
48,88e82447183b34c14c398690db205314,6406538,2076.501953,1.0,0.933894,0.925481


In [28]:
bdf = pd.DataFrame(list(map(lambda x: x.get_data(), best)))
bdf = bdf.drop(columns=['matrix', 'operations'])
bdf.sort_values(by=["test_accuracy"], ascending=False)

,hash,parameters,train_time,train_accuracy,valid_accuracy,test_accuracy
5,a05c0efdac251392b3299fdadd52ccc1,31389066,3807.322998,1.0,0.945212,0.944712
8,1cd3075dacd68bc6fd43ba6d2870a431,23131530,3159.622070,1.0,0.945713,0.944311
3,3ddfed42fa77b6813ef9c2998e6afef3,31389066,4100.892090,1.0,0.945613,0.942708
9,5ca7fb6ce30c7e3ed0ac832ca6431d5d,31389066,3731.862061,1.0,0.948117,0.941707
7,324a2b403f2d571680694086ff4937d2,23131530,3451.044922,1.0,0.944812,0.939503
0,b31b54c1ca6989b1ec51a1fac4fedc27,22257802,3223.629883,1.0,0.943209,0.939303
2,c19d402b167416d7d3853f1f6788fcf3,23131530,3444.309082,1.0,0.945012,0.939002
6,4384e75a71f24965990ef2dd5602155f,33300362,4191.884766,1.0,0.942809,0.939002
4,c90994ab1c1f30967e897f1fa84e27b5,23131530,2952.353027,1.0,0.941907,0.938401
1,dbbee9e11fd122b21ef2ecb865c34e81,23131530,3153.926025,1.0,0.939303,0.934896
